In [38]:
import pandas as pd
import numpy as np
import pickle

# Read the data
imp6 = pd.read_csv("../data.csv")

# Analysis on important votes
X5 = imp6.drop(imp6.columns[0], axis=1)

# Abstain as agree
X55 = X5[X5['importantvote'] == 1]
X56 = X55.drop(X55.columns[15:19], axis=1)
X57 = X56.drop(X55.columns[23:31], axis=1)
X66 = X57.dropna()

print(len(np.unique(X66['rcid'])))
## Note that the total number of important votes in the original data between 2001-2015 is 162
print(len(np.unique(X66['country'])))
print(np.unique(X66['year']))

# Index
year = X66.iloc[:,0]
country = X66.iloc[:,1]

# DVs
# the order is: US reward, US panish, comply, increase
## Two types of choices: abstain as agree or disagree; no CHN aid recipients as not receiving increase from China or missingness? namely, increaseA or increase
Y82 = X66.iloc[:, [13, 14, 4, 16]]
X7 = X66.drop(X66.columns[range(0,22)], axis=1)

# Abstain as non-compliance
colnc = [71, 67, 58, 63, 64, 68, 69,70, 72, 76, 77, 87]
colnr = [47,43, 34, 39, 40, 44,45,46, 50, 52, 53, 0, 3, 4]
colnu = [23, 19, 10, 15, 16, 21, 22, 26, 28, 29, 79, 0, 3, 4]
collistDA = [colnc, colnc, colnc, colnc, colnr, colnr, colnr, colnr, colnu, colnu, colnu, colnu]

import numpy as np
from scipy.stats import norm
import pandas as pd

def BTnormlog(x, mu, sigma=1):
    if x==0:
        u = np.random.uniform()
        xstar = norm.ppf(np.log(u) + norm.logpdf(0, mu, sigma))
    else:
        u = np.random.uniform()
        xstar = -norm.ppf(np.log(u) + norm.logpdf(0, -mu, sigma))
    return xstar

161
123
[2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014
 2015]


In [104]:
def GameMCMC12(Y, X, year, country, covset, m=10000, burnin=5000, h=0.001):
    N = len(Y)
    
    ci1, ci2, ci3, ci4 = covset[:4]
    rc1, rc2, rcn1, rcn2 = covset[4:8]
    ur1, ur2, up1, up2 = covset[8:]

    k1, k2, k3, k4 = len(ci1), len(ci2), len(ci3), len(ci4)
    k5, k6, k7, k8 = len(rc1), len(rc2), len(rcn1), len(rcn2)
    k9, k10, k11, k12 = len(ur1), len(ur2), len(up1), len(up2)

    ## US reward given Recipient comply
    usr = Y.iloc[:,2] == 1
    USReward = Y[usr].iloc[:,0]
    Xu7R = X[usr].iloc[:,ur1]
    Xu9R = X[usr].iloc[:,ur2]
    year_usr = year[usr]
    country_usr = country[usr]
    Nusreward = len(usr)

    ## US punish given Recipient not comply
    usp = Y.iloc[:,2]==0
    USPunish = Y[usp].iloc[:,1]
    Xu12P = X[usp].iloc[:,up1]
    Xu14P = X[usp].iloc[:,up2]
    year_usp = year[usp]
    country_usp = country[usp]
    Nuspunish = len(usp)

    # Recipient comply or not given China increase
    rci = Y.iloc[:,3]==1
    RComply = Y[rci].iloc[:,2]
    Xr7C = X[rci].iloc[:,rc1]
    Xr12C = X[rci].iloc[:,rc2]
    year_rci = year[rci]
    country_rci = country[rci]
    Nrcomply = len(rci)

    # Recipient comply or not given China not increases
    rcni = Y.iloc[:,3]==0
    RNComply = Y[rcni].iloc[:,2]
    Xr9NC = X[rcni].iloc[:,rcn1]
    Xr14NC = X[rcni].iloc[:,rcn2]
    year_rcni = year[rcni]
    country_rcni = country[rcni]
    Nrncomply = len(rcni)

    # China increase or not
    CIncrease = Y.iloc[:,3]
    Xc7 = X.iloc[:,ci1]
    Xc8 = X.iloc[:,ci2]
    Xc11 = X.iloc[:,ci3]
    Xc12 = X.iloc[:,ci4]
    ucountry_ci = np.unique(country)
    uyear_ci = np.unique(year)
    N_country = len(ucountry_ci)
    T_year = len(uyear_ci)

    betac7 = np.zeros(k1)
    betac8 = np.zeros(k2)
    betac11 = np.zeros(k3)
    betac12 = np.zeros(k4)

    betar7 = np.zeros(k5)
    betar9 = np.zeros(k6)
    betar12 = np.zeros(k7)
    betar14 = np.zeros(k8)

    betau7 = np.zeros(k9)
    betau9 = np.zeros(k10)    
    betau12 = np.zeros(k11)
    betau14 = np.zeros(k12)

    betar8 = 0
    betar10 = 0
    betac14 = 0 # intercept is put in the place of (-increase, not compliance, punish)

    p1 = p2 = p3 = p4 = p5 = p6 = p7 = p8 = p9 = p10 = p11 = p12 = p13 = p14 = 0
    p7rc = p7rnc = p12rc = p12rnc = p8rc = p9rnc = p14rnc = p10rnc = 0

    # US reward given compliance
    ones_col = np.ones((Xu7R.shape[0],1))
    Xuc = np.hstack((ones_col, p1*Xu7R, p2*Xu9R))
    betaucr = np.concatenate((np.array([[0]]), betau7.reshape(-1,1), betau9.reshape(-1,1)), axis=0)
    newUusr = Xuc @ betaucr

    # US Punish given non-compliance
    ones_col = np.ones((Xu12P.shape[0],1))
    Xunc = np.hstack((ones_col, p1*Xu12P, p2*Xu14P))
    betauncp = np.concatenate((np.array([[0]]), betau12.reshape(-1,1), betau14.reshape(-1,1)), axis=0)
    newUusp = Xunc @ betauncp

    # Recipient Comply given China increases
    col = np.full((Xr7C.shape[0],1), p8)
    Xrc = np.hstack((p7*Xr7C, col, -p12 * Xr12C))
    betarc = np.concatenate((betar7, np.array([betar8]),betar12), axis=0)
    newUrc = Xrc @ betarc

    # Recipient Not Comply
    
    #Xrnc <- cbind(p9*Xr9NC, p10, -p14*Xr14NC)
    col = np.full((Xr9NC.shape[0],1), p10)
    Xrnc = np.hstack((p9*Xr9NC, col, -p14*Xr14NC))
    #betarnc <- c(betar9, betar10, betar14)
    betarnc = np.concatenate((betar9, np.array([betar10]), betar14), axis=0)
    #newUrnc <- as.matrix(Xrnc)%*% as.matrix(betarnc)
    newUrnc = Xrnc @ betarnc

    # China Increase
    # Xci <- cbind(p3*p7*Xc7, p3*p8*Xc8,
    #              p4*p11*Xc11, p4*p12*Xc12, -p6*p14)
    col = np.full((Xc7.shape[0], 1), -p6*p14)
    Xci = np.hstack((p3*p7*Xc7, p3*p8*Xc8, p4*p11*Xc11, p4*p12*Xc12, col))
    # Íbetaci <- c(betac7, betac8, betac11, betac12, betac14)
    Ibetaaci = np.concatenate((betac7, betac8, betac11, betac12, np.array([betac14])), axis=0)
    # newUci <- as.matrix(Xci)%*% as.matrix(betaci)
    newUci = Xci @ Ibetaaci




GameMCMC12(Y=Y82, X=X7, year=year, country=country, covset=collistDA, m=10000, burnin=5000, h=0.001) 